In [5]:
import pandas as pd
import joblib
import japanize_matplotlib
from preprocessing.initial_preprocessing import initial_preprocessing
from preprocessing.drop_emergency_suspicious_height_weight import drop_emergency_suspicious_height_weight
from preprocessing.remove_duplicate_ctdi import remove_duplicate_ctdi
from preprocessing.rename_and_drop_order import rename_and_drop_order_one_scan_ctdi
from preprocessing.remove_not_use_head_rest import remove_not_use_head_rest
from preprocessing.special_scan_exclude import special_scan_exclude
from feature_engineering.dose_prediction_feature_engineering import FeatureEngineering

In [ ]:
# テストデータの読み込み
df = pd.read_excel('test_data.xlsx')

In [ ]:
# データの前処理
initial_preprocesing(df)
df = remove_duplicate_ctdi(df)
df.reset_index(drop=True, inplace=True)
df_all = df
df = drop_emergency_suspicious_height_weight(df)

unuse_cols = ['hospital_ward', 'modality', 'adult_child', 'department', 'preset_name']
df.drop(labels=unuse_cols, axis=1, inplace=True)

df = df[~((df['CTDIw phantom type'] == 'IEC Head Dosimetry Phantom') & (df['scan_area'] == '頸部CT'))]
df = df[~((df['CTDIw phantom type'] == 'IEC Head Dosimetry Phantom') & (df['scan_area'] == '頸部〜骨盤CT'))]
df = df[~((df['CTDIw phantom type'] == 'IEC Head Dosimetry Phantom') & (df['scan_area'] == '胸部〜骨盤CT'))]
df = df[~((df['CTDIw phantom type'] == 'IEC Head Dosimetry Phantom') & (df['scan_area'] == '胸部CT'))]

# スキャンオーダーの名前を変更する
rename_and_drop_order_one_scan_ctdi(df)
# 特殊なスキャン名を取り除く
df = special_scan_exclude(df)

In [ ]:
# テストデータのプロトコールの割合を管電圧の確認
scan_area_list = df['scan_area'].unique()
for scan_area in scan_area_list:
    print(f'{scan_area}:',  df[df['scan_area'] == scan_area]['kV'].value_counts())

### Feature-engineering

In [ ]:
fe = joblib.load('feature_engineering.pkl')
df_test_transformed = fe.transform(df)